In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:

spark = SparkSession.builder.appName('flights').getOrCreate()

22/12/21 11:52:02 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.156 instead (on interface en0)
22/12/21 11:52:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 11:52:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/21 11:52:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
def count_null(df):
    cols_check = df.columns
    df.select(*[
    (
        F.count(F.when((F.isnan(c) | F.col(c).isNull()), c)) if t not in ("timestamp", "date")
        else F.count(F.when(F.col(c).isNull(), c))
    ).alias(c)
    for c, t in df.dtypes if c in cols_check
]).show()

In [4]:
def select_features():
    features = []
    with open("features_analysis.txt","r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line=="":
                continue
            split = line.split()
            feature = split[0].strip()
            features.append(feature)
    return features


In [5]:
def clean_data(df,features):
    df = df.select(features)
    df = df.na.drop()
    return df

In [6]:
def join_airports(df,airports):
    df = df.join(airports,df.Origin==airports.IATA)
    df = df.withColumnRenamed("LATITUDE","ORIGIN_LATITUDE")
    df = df.withColumnRenamed("LONGITUDE","ORIGIN_LONGITUDE")
    df = df.drop("IATA","AIRPORT","CITY","STATE","COUNTRY")
    df = df.join(airports,df.Dest==airports.IATA)
    df = df.withColumnRenamed("LATITUDE","DEST_LATITUDE")
    df = df.withColumnRenamed("LONGITUDE","DEST_LONGITUDE")
    df = df.drop("IATA","AIRPORT","CITY","STATE","COUNTRY")
    return df

In [7]:
def preprocess_normal_data(path):
    df = spark.read.csv(path, inferSchema=True, header=True)
    columns = df.columns
    df_normal = df.filter(df['Diverted'] == 0)
    features = select_features()
    df_normal = clean_data(df_normal,features)

    airports = spark.read.csv("airports.csv", inferSchema=True, header=True)
    df_normal = join_airports(df_normal,airports)
    return df_normal

In [8]:
df_cleaned = preprocess_normal_data("data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2013_12.csv")


In [10]:
# save df_cleaned in a single csv file
df_cleaned.toPandas().to_csv("data/df_cleaned.csv",index=False)

/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

